# Databrick Batch Processing Notebook
This ntoebook was opriginally written in Databricks to the perform the following tasks:
    1. Mount an S3 Bucket and retrieve the data
    2. Clean the data
    3. Query the data
    

# Mount S3 Bucket and Get Data

This section of the notebook retrieves the necessary authentication credentials, mounts the Amazon S3 bucket and imports the data into the appropriate dataframes

In [0]:
# Import Statements
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Retrieve Amazon Credentials
# Specify file type to be csv
file_type = "csv"

# Indicates file has first row as the header
first_row_is_header = "true"

# Indicates file has comma as the delimeter
delimiter = ","

# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']

# Encode the secret key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a25072a5e0f-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0a25072a5e0f-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

# list the topics stored on the mounted S3 bucket
#display(dbutils.fs.ls("/mnt/user-0a25072a5e0f-bucket/topics"))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2509016723051128> in <module> 
 33 SOURCE_URL = "s3n://{0}:{1}@{2}" . format ( ACCESS_KEY , ENCODED_SECRET_KEY , AWS_S3_BUCKET ) 
 34 # Mount the drive 
 ---> 35 dbutils . fs . mount ( SOURCE_URL , MOUNT_NAME ) 
 36 
 37 # list the topics stored on the mounted S3 bucket 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o419.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/user-0a25072a5e0f-bucket; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/user-0a25072a5e0f-bucket
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/user-0a25072a5e0f-bucket
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:656)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$2(MetadataManager.scala:1032)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:805)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:1021)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:664)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:128)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.$anonfun$applyOrElse$9(DbfsServerBackend.scala:386)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:22)

In [0]:
# specify file type
file_type = "json"
# infer the schema
infer_schema = "true"
# specify topic paths
pin_path = "/mnt/user-0a25072a5e0f-bucket/topics/0a25072a5e0f.pin/partition=0/*.json"
geo_path = "/mnt/user-0a25072a5e0f-bucket/topics/0a25072a5e0f.geo/partition=0/*.json"
user_path = "/mnt/user-0a25072a5e0f-bucket/topics/0a25072a5e0f.user/partition=0/*.json"

pin_df = spark.read.format(file_type) \
        .option("inferSchema", "true") \
        .load(pin_path)

geo_df = spark.read.format(file_type) \
        .option("inferSchema", "true") \
        .load(geo_path)

user_df = spark.read.format(file_type) \
        .option("inferSchema", "true") \
        .load(user_path)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-253566981726783> in <module> 
 8 user_path = "/mnt/user-0a25072a5e0f-bucket/topics/0a25072a5e0f.user/partition=0/*.json" 
 9 
 ---> 10 pin_df = spark . read . format ( file_type ) \ 
 11 . option ( "inferSchema" , "true" ) \ 
 12 . load ( pin_path ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 156 self . options ( ** options ) 
 157 if isinstance ( path , str ) : 
 --> 158 return self . _df ( self . _jreader . load ( path ) ) 
 159 elif path is not None : 
 160 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o1543.load.
: com.databricks.backend.daemon.data.common.InvalidMountException: The backend could not get session tokens for path /mnt/user-0a25072a5e0f-bucket/topics/0a25072a5e0f.pin/partition=0. Did you remove the AWS key for the mount point?
	at com.databricks.backend.daemon.data.common.InvalidMountException$.apply(DataMessages.scala:678)
	at com.databricks.backend.daemon.data.filesystem.MountEntryResolver.resolve(MountEntryResolver.scala:96)
	at com.databricks.backend.daemon.data.client.DBFSV2.resolve(DatabricksFileSystemV2.scala:81)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:93)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:70)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.

In [0]:
# Unmount the bucket from the filestore
dbutils.fs.unmount("/mnt/user-0a25072a5e0f-bucket")

/mnt/user-0a25072a5e0f-bucket has been unmounted.
Out[15]: True

# Clean the Data

This section of the notebook cleans the imported data

In [0]:
# clean df_pin
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, regexp_replace, when, substring_index, expr
from pyspark.sql.types import IntegerType

def replace_with_null(dataframe, column, values_to_replace):
    dataframe = dataframe.withColumn(column, when(col(column).like(value_to_replace), None).otherwise(col(column)))
    return dataframe

# replace empty entries in each column
values_to_replace = {
    "description": "No description available%",
    "follower_count": "User Info Error",
    "image_src": "Image src error.",
    "poster_name": "User Info Error",
    "tag_list": "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",
    "title": "No Title Data Available"
}

# loop to find values to replace
for key, value in values_to_replace.items():
    df_pin = replace_with_null(df_pin, key, value)

# remove duplicate rows
df_pin = df_pin.dropDuplicates()

# deal with follower_count values to ensure they are ints
df_pin = df_pin.withColumn(
    "follower_count_numeric",
    (
        regexp_extract("follower_count", "(\d+)([kKmM]?)", 1)
        .cast("double")
        * when(
            (col("follower_count").rlike("[Mm]")), 1000000
        ).when(
            (col("follower_count").rlike("[kK]")), 1000
        ).otherwise(1)
    ).cast("int")
)

# drop the original follower_count column
df_pin = df_pin.drop("follower_count")

# rename the new column to follower_count
df_pin = df_pin.withColumnRenamed("follower_count_numeric", "follower_count")

# clean the data in the save_location column to include only the save location path
column_to_transform = "save_location"
df_pin = df_pin.withColumn(
    column_to_transform,
    regexp_replace(col(column_to_transform), "^Local save in ", "")
)

# rename the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")

# reorder the dataframe columns
column_order = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
df_pin = df_pin.select(column_order)

In [0]:
# clean df_geo
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, array, to_timestamp

# remove duplicate rows
df_geo = df_geo.dropDuplicates()

# create a new column 'coordinates' that contains an array based on the latitude and longitude columns
df_geo = df_geo.withColumn('coordinates', array('latitude', 'longitude'))

# drop the 'latitude' and 'longitude' columns form the dataframe
df_geo = df_geo.drop('latitude', 'longitude')

# convert the 'timestamp' column from string to timestamp data type
df_geo = df_geo.withColumn('timestamp', to_timestamp(col('timestamp')))

# rename index column
df_geo = df_geo.withColumnRenamed("index", "ind")

# reorder the dataframe columns
column_order = ['ind', 'country', 'coordinates', 'timestamp']
df_geo = df_geo.select(column_order)

In [0]:
# clean df_user
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, to_timestamp

# remove duplicate rows
df_user = df_user.dropDuplicates()

# create a new column user_name that concatenates the information found in the first_name and last_name columns
df_user = df_user.withColumn('user_name', concat_ws('', 'first_name', 'last_name'))

# drop the 'first_name' and 'last_name' columns from dataframe
df_user = df_user.drop('first_name', 'last_name')

# convert the 'date_joined' column from string to timestamp
df_user = df_user.withColumn('date_joined', to_timestamp(col('date_joined')))

# rename index column
df_user = df_user.withColumnRenamed("index", "ind")

# reorder the dataframe columns
column_order = ['ind', 'user_name', 'age', 'date_joined']
df_user = df_user.select(column_order)


In [0]:
# check schemas
df_pin.printSchema()
df_geo.printSchema()
df_user.printSchema()

# Query the Data

This section of the notebook conducts the necessary queries on the data

In [0]:
# find the most popular Pinterest category people post to based on their country

# register dataframes as temporary tables
df_geo.createOrReplaceTempView("geo_table")
df_pin.createOrReplaceTempView("pin_table")

# use SQL to query
query_df = spark.sql("""
    SELECT DISTINCT 
        geo_table.country, 
        pin_table.category, 
        count(pin_table.category) as category_count
    FROM 
        geo_table
    INNER JOIN 
        pin_table ON geo_table.ind = pin_table.ind
    GROUP BY 
        geo_table.country, 
        pin_table.category 
""")

display(query_df)

In [0]:
# find how many posts each category had between 2018 and 2022

# register dataframes as temporary tables
df_geo.createOrReplaceTempView("geo_table")
df_pin.createOrReplaceTempView("pin_table")

# use SQL to query
query_df = spark.sql("""
    SELECT DISTINCT 
        YEAR(geo_table.timestamp) AS post_year, 
        pin_table.category, 
        COUNT(pin_table.category) AS category_count
    FROM 
        geo_table
    INNER JOIN 
        pin_table ON geo_table.ind = pin_table.ind
    WHERE 
        YEAR(geo_table.timestamp) >= 2018 AND YEAR(geo_table.timestamp) <= 2022
    GROUP BY 
        post_year, pin_table.category
""")

display(query_df)

In [0]:
# for each country find the user with the most followers

# register dataframes as temporary tables
df_geo.createOrReplaceTempView("geo_table")
df_pin.createOrReplaceTempView("pin_table")

# use SQL to query
query_df = spark.sql("""
                      
    WITH RankedResults AS (
        SELECT DISTINCT
            geo_table.country AS country,
            pin_table.poster_name AS poster_name,
            pin_table.follower_count AS follower_count,
            ROW_NUMBER() OVER (PARTITION BY geo_table.country ORDER BY pin_table.follower_count DESC) AS row_num
        FROM
            geo_table
        INNER JOIN pin_table ON geo_table.ind = pin_table.ind
        )
    SELECT
        country,
        poster_name,
        follower_count
    FROM
        RankedResults
    WHERE
        row_num = 1;
""")

display(query_df)

# based on the above query, find the country with the user with most followers

# register dataframes as temporary tables
query_df.createOrReplaceTempView("result_table")

# use SQL to query
query_2_df = spark.sql("""
    SELECT 
        country, follower_count
    FROM 
        result_table
    ORDER BY 
        follower_count DESC
    LIMIT 1;
""")

display(query_2_df)


In [0]:
# what is the most popular category people post to based on the following age groups: 18-24, 25-35, 36-50, +50

# register dataframes as temporary tables
df_user.createOrReplaceTempView("user_table")
df_pin.createOrReplaceTempView("pin_table")

# use SQL to query
query_df = spark.sql("""
    WITH age_group_table AS (
        SELECT
            ind,
            CASE
                WHEN age BETWEEN 18 AND 24 THEN '18-24'
                WHEN age BETWEEN 25 AND 35 THEN '25-35'
                WHEN age BETWEEN 36 AND 50 THEN '36-50'
                ELSE '+50'
            END AS age_group
        FROM
            user_table
    )

    SELECT
        age_group_table.age_group,
        pin_table.category,
        COUNT(DISTINCT age_group_table.ind, pin_table.category) AS category_count
    FROM
        age_group_table
    JOIN
        pin_table ON age_group_table.ind = pin_table.ind
    GROUP BY
        age_group_table.age_group, 
        pin_table.category
    ORDER BY
        age_group_table.age_group, 
        category_count DESC;
""")

display(query_df)

In [0]:
# what is the median follower count for users in the following age groups: 18-24, 25-35, 36-50, +50

# register dataframes as temporary tables
df_user.createOrReplaceTempView("user_table")
df_pin.createOrReplaceTempView("pin_table")

# use SQL to query
query_df = spark.sql("""
    WITH age_group_table AS (
        SELECT
            ind,
            CASE
                WHEN age BETWEEN 18 AND 24 THEN '18-24'
                WHEN age BETWEEN 25 AND 35 THEN '25-35'
                WHEN age BETWEEN 36 AND 50 THEN '36-50'
                ELSE '+50'
            END AS age_group
        FROM
            user_table
    )

    SELECT
        age_group_table.age_group,
        percentile_approx(pin_table.follower_count, 0.5) AS median_follower_count
    FROM
        age_group_table
    JOIN
        pin_table ON age_group_table.ind = pin_table.ind
    GROUP BY
        age_group_table.age_group
    ORDER BY
        median_follower_count DESC;
""")

display(query_df)

In [0]:
# find how many users have joined between 2015 and 2020

# register dataframes as temporary tables
df_user.createOrReplaceTempView("user_table")

# use SQL to query
query_df = spark.sql("""
    SELECT
        YEAR(user_table.date_joined) AS post_year,
        COUNT(DISTINCT(ind)) AS number_users_joined
    FROM
        user_table
    GROUP BY
        post_year
    ORDER BY
        post_year DESC;
""")

display(query_df)

In [0]:
# find the median follower count of users have joined between 2015 and 2020

# register dataframes as temporary tables
df_user.createOrReplaceTempView("user_table")
df_pin.createOrReplaceTempView("pin_table")

# use SQL to query
query_df = spark.sql("""
    SELECT
        YEAR(user_table.date_joined) AS post_year,
        percentile_approx(pin_table.follower_count, 0.5) AS median_follower_count
    FROM
        user_table
    JOIN 
        pin_table ON user_table.ind = pin_table.ind
    GROUP BY
        post_year
    ORDER BY
        post_year DESC;
""")

display(query_df)

In [0]:
# find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of

# register dataframes as temporary tables
df_user.createOrReplaceTempView("user_table")
df_pin.createOrReplaceTempView("pin_table")

# use SQL to query
query_df = spark.sql("""
    WITH age_group_table AS (
        SELECT
            ind,
            CASE
                WHEN age BETWEEN 18 AND 24 THEN '18-24'
                WHEN age BETWEEN 25 AND 35 THEN '25-35'
                WHEN age BETWEEN 36 AND 50 THEN '36-50'
                ELSE '+50'
            END AS age_group
        FROM
            user_table
    )
    SELECT
        age_group_table.age_group AS age_group,
        YEAR(user_table.date_joined) AS post_year,
        percentile_approx(pin_table.follower_count, 0.5) AS median_follower_count
    FROM
        user_table
    JOIN 
        pin_table ON user_table.ind = pin_table.ind 
    JOIN 
        age_group_table ON user_table.ind = age_group_table.ind
    GROUP BY
        age_group, post_year
    ORDER BY
        post_year ASC, age_group ASC;
""")

display(query_df)